In [1]:
#import sys

#!{sys.executable} -m pip install farm-haystack -f https://download.pytorch.org/whl/torch_stable.html
#!{sys.executable} -m pip install farm-haystack[ocr]

Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached farm_haystack-1.9.1-py3-none-any.whl (733 kB)
  Using cached elasticsearch-7.10.1-py2.py3-none-any.whl (322 kB)
  Using cached more_itertools-8.14.0-py3-none-any.whl (52 kB)
  Using cached dill-0.3.5.1-py2.py3-none-any.whl (95 kB)
  Using cached python-docx-0.8.11.tar.gz (5.6 MB)
  Using cached tika-1.24.tar.gz (28 kB)
  Using cached quantulum3-0.7.10-py3-none-any.whl (10.7 MB)
  Using cached elastic-apm-6.12.0.tar.gz (174 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached mlflow-1.29.0-py3-none-any.whl (16.9 MB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Using cached sentence-t

ERROR: Invalid requirement: "'farm-haystack[ocr]'"


In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [4]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


In [5]:
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http
from haystack.nodes import PreProcessor


# Let's first get some documents that we want to query
# Here: 517 Wikipedia articles for Game of Thrones
doc_dir = "Datasets/Watchman"

#s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip"
#fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

# convert files to dicts containing documents that can be indexed to our datastore
# You can optionally supply a cleaning function that is applied to each doc (e.g. to remove footers)
# It must take a str as input, and return a str.
all_docs = convert_files_to_docs(dir_path=doc_dir, clean_func = lambda x:x.replace("\n",""), split_paragraphs=True)

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=200,
    split_overlap=30,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

# We now have a list of dictionaries that we can write to our document store.
# If your texts come from a different source (e.g. a DB), you can of course skip convert_files_to_dicts() and create the dictionaries yourself.
# The default format here is: {"name": "<some-document-name>", "content": "<the-actual-text>"}

# Let's have a look at the first 3 entries:
print(docs[:3])

# Now, let's write the docs to our DB.
document_store.write_documents(docs)

INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\1-s2.0-S0002870317300881-main.pdf
INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\Alternative to Warfarin – WATCHMAN Implant.pdf
INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\How The WATCHMAN Device Works.pdf
INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\WATCHMAN Implant for Non-Valvular Afib Stroke Risk.pdf
INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\WATCHMAN Procedure vs. Blood Thinners.pdf
INFO - haystack.utils.preprocessing -  Converting Datasets\Watchman\WATCHMAN_DTP_Brochure.pdf


Preprocessing:   0%|          | 0/6 [00:00<?, ?docs/s]

WARNING - haystack.nodes.preprocessor.preprocessor -  One or more sentence found with word count higher than the split length.


n_files_input: 6
n_docs_output: 97
[<Document: {'content': 'The Assessment of the Watchman Device inPatients Unsuitable for Oral Anticoagulation(ASAP-TOO) trialDavid R. Holmes, MD, aVivek Y. Reddy, MD, bMaurice Buchbinder, MD, cKenneth Stein, MD, dMyriah Elletson dMartin W. Bergmann, MD, eBoris Schmidt, MD, fand Jacqueline Saw, MD, FRCPC gRochester, Minneapolis, MN;New York, NY; Stanford, CA; Hamburg, Frankfurt, Germany; and British Columbia, CanadaBackground Oral anticoagulants (OACs) reduce stroke risks with nonvalvular atrial fibrillation (AF); however, they areunderused because of absolute or relative contraindications due to real or perceived risk of bleeding. Although left atrialappendage closure is increasingly performed in OAC-ineligible patients, this has not been studied in a randomizedcontrolled trial.Study objectives The ASAP-TOO study is designed to establish the safety and effectiveness of the Watchman leftatrial appendage closure device in patients with nonvalvular AF wh

Testing with InMemoryDocumentStore used TfidfRetriever, when use Elasticsearch can use BM25Retriever

In [6]:
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store=document_store)

INFO - haystack.nodes.retriever.sparse -  Found 97 candidate paragraphs from 97 docs in DB


In [7]:
from haystack.nodes import FARMReader


# Load a  local model or any of the QA models on
# Hugging Face's model hub (https://huggingface.co/models)
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 11 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\   /w\   /w\   /w\ 
INFO - haystack.modeling.infer -  /'\   / \   /'\   /'\   / \   / \   /'\   /'\   /'\   /'\   /'\ 


In [8]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [9]:
# You can configure how many candidates the reader and retriever shall return
# The higher top_k for retriever, the better (but also the slower) your answers.
prediction = pipe.run(
    query="What are the risks of having a Watchman implant procedure?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 10.00 Batches/s]


In [10]:
# Now you can either print the object directly...
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'accidental heart puncture, air embolism, allergic reaction, anemia,anesthesia risks', 'type': 'extractive', 'score': 0.5987601280212402, 'context': 'sks include but are not limited toaccidental heart puncture, air embolism, allergic reaction, anemia,anesthesia risks, arrhythmias, AV (Arteriovenous)', 'offsets_in_document': [{'start': 630, 'end': 713}], 'offsets_in_context': [{'start': 34, 'end': 117}], 'document_id': '6b7c6ac4c46533c0279ea119cd53a8c3', 'meta': {'name': 'WATCHMAN_DTP_Brochure.pdf', '_split_id': 17}}>,
             <Answer {'answer': 'stroke risk and bleeding worry for life', 'type': 'extractive', 'score': 0.44617047905921936, 'context': 'is a one‑time, minimally invasive procedure thatreduces stroke risk and bleeding worry for life.One Time. For a Lifetime.AND COUNTINGLIVES CHANGEDTwee', 'offsets_in_document': [{'start': 305, 'end': 344}], 'offsets_in_context': [{'start': 56, 'end': 95}], 'document_id': '3a9ec309fb970959a3fe34e6cdf95a24'

In [11]:
# ...or use a util to simplify the output
from haystack.utils import print_answers


# Change `minimum` to `medium` or `all` to control the level of detail
print_answers(prediction, details="medium")


Query: What are the risks of having a Watchman implant procedure?
Answers:
[   {   'answer': 'accidental heart puncture, air embolism, allergic reaction, '
                  'anemia,anesthesia risks',
        'context': 'sks include but are not limited toaccidental heart '
                   'puncture, air embolism, allergic reaction, '
                   'anemia,anesthesia risks, arrhythmias, AV (Arteriovenous)',
        'score': 0.5987601280212402},
    {   'answer': 'stroke risk and bleeding worry for life',
        'context': 'is a one‑time, minimally invasive procedure thatreduces '
                   'stroke risk and bleeding worry for life.One Time. For a '
                   'Lifetime.AND COUNTINGLIVES CHANGEDTwee',
        'score': 0.44617047905921936},
    {   'answer': 'patients should not be considered for the WATCHMAN Implant '
                  'ifthey are doing well and expect to continue doing well on '
                  'blood thinners',
        'context': 'ocedure, p